In [0]:
from google.colab import drive
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [0]:
import tensorflow as tf
from tensorflow import keras
import torch
import glob
import numpy as np
import cv2
import pandas as pd
from torch import nn
from torch.utils import data
from tqdm.notebook import tqdm
from PIL import Image
from torchvision.transforms import ToTensor
import torchvision.models as models
import torch.optim as optim
import os
os.chdir("/content/gdrive/My Drive/CS543/test/")

In [0]:
#!unzip vid6.zip -d data

In [0]:
DATASET_PATH = 'data/colored_data/'
csv_path = 'data/colored_data/newAnnotations.csv'
annotations = pd.read_csv(csv_path)
LABELS = ['yield', 'keepRight', 'noLeftTurn', 'speedLimitUrdbl', 'speedLimit25', 'stopAhead', 'stop', 'merge', 'speedLimit65', 'truckSpeedLimit55', 'rampSpeedAdvisory45', 'signalAhead', 'noRightTurn', 'speedLimit50', 'pedestrianCrossing', 'speedLimit35', 'dip', 'schoolSpeedLimit25', 'school', 'yieldAhead', 'addedLane', 'laneEnds', 'rampSpeedAdvisoryUrdbl', 'rampSpeedAdvisory20', 'zoneAhead45', 'rightLaneMustTurn', 'rampSpeedAdvisory50', 'speedLimit40', 'speedLimit45', 'turnRight', 'speedLimit55', 'slow', 'turnLeft', 'doNotPass', 'intersection']
EXTRACTED = annotations["Filename;Annotation tag;Upper left corner X;Upper left corner Y;Lower right corner X;Lower right corner Y;Occluded"]

In [0]:
class LISADataset(data.Dataset):
    def __init__(self, split = "train" ,data_dir=DATASET_PATH, annotations = EXTRACTED): 
        assert(split in ["train", "val", "test"])
        self.img_dir = DATASET_PATH #os.path.join(data_dir, split)
        self.labels = LABELS
        self.annotations = EXTRACTED
        self.imgs = []
        self.boxes = {}
        self.labels = {}

        for i in self.annotations:
          name, label, x1, y1, x2, y2, *_ = i.split(';')
          temp = [int(x1), int(y1), int(x2), int(y2)]
          name = name.split('/')[2]
          self.imgs.append(name)

          if name in self.boxes.keys():
            self.boxes[name].append(temp)
            self.labels[name].append(LABELS.index(label))
          else:
            self.boxes[name] = [temp]
            self.labels[name] = [LABELS.index(label)]
        self.n_classes = len(self.labels)
    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, index):
        img = Image.open(DATASET_PATH + self.imgs[index])
        img = ToTensor()(img)
        box = torch.as_tensor(self.boxes[self.imgs[index]],dtype=torch.float32)
        label = torch.as_tensor(self.labels[self.imgs[index]])
        return img, box, label

In [0]:
##load dataset and split for train and test
dataset = LISADataset()
train, test = torch.utils.data.random_split(dataset, [2900,332])
def collate_fn(batch):
    return zip(*batch)
train_loader = data.DataLoader(train, batch_size=8, shuffle=True, num_workers=4, collate_fn= collate_fn)
test_loader = data.DataLoader(test, batch_size=1, shuffle=True, num_workers=4, collate_fn= collate_fn)

In [0]:
print("Length of training set:",len(train))
print("Length of testing set:",len(test))

Length of training set: 2900
Length of testing set: 332


In [0]:
torch.cuda.empty_cache()
model = models.detection.fasterrcnn_resnet50_fpn(pretrained=False,num_classes=len(LABELS))
model.cuda()

AttributeError: ignored

In [0]:
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.7)

In [0]:
# training
train_loss_over_epochs = []
EPOCHS = 10
for epoch in tqdm(range(EPOCHS), total=EPOCHS):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, batch in enumerate(tqdm(train_loader)):
        # get the inputs
        img, box, label = batch
        optimizer.zero_grad()
        # forward + backward + optimize
        images = list(image.cuda() for image in img)
        targets = []
        for i in range(len(images)):
            d = {}
            d['boxes'] = box[i].cuda()
            d['labels'] = label[i].cuda()
            targets.append(d)
        outputs = model(images, targets)
        loss = outputs['loss_classifier'].mean() + outputs['loss_box_reg'].mean() + outputs['loss_objectness'].mean() + outputs['loss_rpn_box_reg'].mean()
        #loss = criterion(outputs, labels.squeeze(1))
        
        loss.backward() 
        optimizer.step()

        # print statistics
        running_loss += loss.item()
    
    # Normalizing the loss by the total number of train batches
    running_loss/=len(train_loader)
    train_loss_over_epochs.append(running_loss)
    print('[%d] loss: %.3f' % (epoch + 1, running_loss))

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "
/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of nonzero is deprecated:
	nonzero(Tensor input, *, Tensor out)
Consider using one of the following signatures instead:
	nonzero(Tensor input, *, bool as_tuple)



[1] loss: 0.474



[2] loss: 0.231



[3] loss: 0.179



[4] loss: 0.162


In [0]:
# ######this step is used to save trained model
# checkpoint = {'model': models.detection.fasterrcnn_resnet50_fpn(pretrained=False,num_classes=len(LABELS)),
#               'state_dict': model.state_dict(),
#               'optimizer' : optimizer.state_dict()}

# torch.save(checkpoint, '30epoch.pth')

In [0]:
import matplotlib.pyplot as plt
plt.ioff()
fig = plt.figure()
plt.ylabel('Train loss')
plt.xlabel('Epochs')
plt.plot(np.arange(EPOCHS), train_loss_over_epochs, 'k-')
plt.title('Train loss vs Epochs')
plt.xticks(np.arange(EPOCHS, dtype=int))
plt.grid(True)
plt.show()

In [0]:
######this is the test code for loading trained model
def load_checkpoint(filepath):
    checkpoint = torch.load(filepath)
    model = checkpoint['model']
    model.load_state_dict(checkpoint['state_dict'])
    for parameter in model.parameters():
        parameter.requires_grad = False
    
    model.eval()
    
    return model

In [0]:
model = load_checkpoint('30epoch.pth')
model.cuda()

In [0]:
#######TESTING PHASE
#predictions is given as a dict
#create new dict for groun truth
from torch.autograd import Variable
with torch.no_grad():
  model.eval()
  gts = []
  preds = []
  for i, batch in enumerate(tqdm(test_loader)):
    img, box, label = batch
    for i in range(len(box)):
      label = label[i].tolist()[0]
      box = box[i].tolist()[0]
      gts.append([label, box])
    images = list(image.cuda() for image in img)
    pred = model(images)[0]
    box_ = pred['boxes']
    label_ = pred['labels']
    score_ = pred['scores']
    for i in range(len(box_)):
      label = label_[i].item()
      box = box_[i].tolist()
      score = score_[i].tolist()
      preds.append([label, box, score])

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "


In [0]:
#######TESTING PHASE
#predictions is given as a dict
#create new dict for groun truth
import torchvision
from torch.autograd import Variable
with torch.no_grad():
  model.eval()
  gts = []
  preds = []
  y_true = []
  y_pred = []
  for i, batch in enumerate(tqdm(test_loader)):
    img, box, label = batch
    count = 0
    for i in range(len(box)):
      label = label[i].tolist()[0]
      box = box[i].tolist()[0]
      gts.append([label, box])
      y_true.append(label)
      count += 1
    images = list(image.cuda() for image in img)
    pred = model(images)[0]
    box_ = pred['boxes']
    label_ = pred['labels']
    score_ = pred['scores']
    iou_th = 0.05
    idx = torchvision.ops.nms(box_,score_,iou_th)
    keep_box = box_[idx]
    keep_score = score_[idx]
    keep_label = label_[idx]
    for i in range(len(keep_box)):
      label = keep_label[i].item()
      box = keep_box[i].tolist()
      score = keep_score[i].tolist()
      preds.append([label, box, score])
    for i in range(count):
      if i < len(keep_label):
        y_pred.append(keep_label[i].tolist())
      else:
        y_pred.append(-1)

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "


In [0]:
print("Number of ground truth labels:", len(gts))
print("Number of predicted labels before NMS:", len(preds))
len(LABELS)

Number of ground truth labels: 332
Number of predicted labels before NMS: 552


35

In [0]:
###sort gts and preds based on label class
#gts = [[label, box],[label, box]]
#preds = [[label, box, score],[label, box, score]]
#gts, preds sorted according to class label
from collections import defaultdict
d = defaultdict(list)
a = defaultdict(list)
for k, *v in gts:
  d[k].append(v[0])
for k, *v in preds:
  a[k].append(v)
#list(d.items())
gts_sorted = sorted(d.items(),key = lambda x:x[0])
preds_sorted = sorted(a.items(),key = lambda x:x[0])

In [0]:
containedClass = []
for i in gts_sorted:
  containedClass.append(i[0])

In [0]:
###Need to add reference for this code
def calc_iou_individual(pred_box, gt_box):
    x1_t, y1_t, x2_t, y2_t = gt_box
    x1_p, y1_p, x2_p, y2_p = pred_box
    if (x1_p > x2_p) or (y1_p > y2_p):
        raise AssertionError(
            "Prediction box is malformed? pred box: {}".format(pred_box))
    if (x1_t > x2_t) or (y1_t > y2_t):
        raise AssertionError(
            "Ground Truth box is malformed? true box: {}".format(gt_box))
    if (x2_t < x1_p or x2_p < x1_t or y2_t < y1_p or y2_p < y1_t):
        return 0.0
    far_x = np.min([x2_t, x2_p])
    near_x = np.max([x1_t, x1_p])
    far_y = np.min([y2_t, y2_p])
    near_y = np.max([y1_t, y1_p])

    inter_area = (far_x - near_x + 1) * (far_y - near_y + 1)
    true_box_area = (x2_t - x1_t + 1) * (y2_t - y1_t + 1)
    pred_box_area = (x2_p - x1_p + 1) * (y2_p - y1_p + 1)
    iou = inter_area / (true_box_area + pred_box_area - inter_area)
    return iou

In [0]:
########calculate precision and recall without NMS
 precisions = {}
 recalls = {}
for gts_class in gts_sorted:
  class_label = gts_class[0]
  gts_box = gts_class[1]
  preds = []
  ##find corresponding class predictions
  for i in range(len(preds_sorted)):
    if preds_sorted[i][0] == class_label:
      preds = preds_sorted[i][1]
      break
  ##current class not presented in preds, precision recall value not included
  if len(preds) == 0:
    continue
  preds = sorted(preds, key = lambda x:x[1], reverse = True)
  #for each ground truth box, find the corresponding matching prediction by greedy approach
  l = len(preds)
  rank = np.zeros(l)
  precision = np.zeros(l)
  recall = np.zeros(l)
  possible = len(gts_box)
  for gt_box in gts_box:
    for i in range(l):  
      iou = calc_iou_individual(preds[i][0], gt_box)
      if iou >= 0.5 and not rank[i]:
        rank[i] = 1
        break
  for i in range(l):
    precision[i] = sum(rank[:(i+1)])/(i+1)
    recall[i] = sum(rank[:(i+1)])/len(gts_box)
      
  precisions[class_label] = precision
  recalls[class_label] = recall
  ##current class presented in preds  

In [0]:
def voc_ap(rec, prec):
    rec = rec.reshape((-1,1))
    prec = prec.reshape((-1,1))
    z = np.zeros((1,1)) 
    o = np.ones((1,1))
    mrec = np.vstack((z, rec, o))
    mpre = np.vstack((z, prec, z))

    mpre = np.maximum.accumulate(mpre[::-1])[::-1]
    I = np.where(mrec[1:] != mrec[0:-1])[0]+1;
    ap = np.sum((mrec[I] - mrec[I-1])*mpre[I])
    return ap

In [0]:
########Need to calculate AP curve for each class and mAP
######results from above: precisions = list of precisions for each calss, recalls = list of recalls for each calss
def compute_map(precisions, recalls):
    aps = []
    for class_label in precisions:
      ap = voc_ap(precisions[class_label], recalls[class_label])
      aps.append(ap)
    return sum(aps)/len(aps)

In [0]:
compute_map(precisions, recalls)

0.25983351853753556

In [0]:
#######compute confusion matrix
import seaborn as sn
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_true, y_pred, labels = containedClass)
# range(len(LABELS))
df_cm = pd.DataFrame(cm, index = range(28), columns =  range(28))
fig, ax = plt.subplots(figsize=(25,25))
sn.heatmap(df_cm, annot = True, annot_kws={"size": 25} ,linewidths=0.5,ax = ax)
plt.show()

In [0]:
#### iou_th = 0.01, 332,466, 0.665249
#### iou_th = 0.01, 332,466, 

In [0]:
######single image inference
import torchvision
for i, batch in enumerate(test_loader):
    img, box, label = batch
    images = list(image.cuda() for image in img)
    pred = model(images)[0]
    break

boxes = pred['boxes']
scores = pred['scores']
labels = pred['labels']
iou_th = 0.01
idx = torchvision.ops.nms(boxes, scores, iou_th)
boxes = boxes[idx]
scores = scores[idx]
labels = labels[idx]

In [0]:
#####plot single inference result
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import numpy as np
temp = images[0].permute((1,2,0))
plt.ioff()
# Create figure and axes
fig,ax = plt.subplots(1)
# Display the image
ax.imshow(temp.cpu().numpy())
for i in range(len(boxes)):
# Create a Rectangle patch
  a = boxes[i][0]
  b = boxes[i][1]
  h = boxes[i][2] - boxes[i][0]
  w = boxes[i][3] - boxes[i][1]
  rect = patches.Rectangle((a,b),h,w,linewidth=1,edgecolor='r',facecolor='none')
  # Add the patch to the Axes
  ax.add_patch(rect)
  break
plt.show()